In [2]:
# import packages 
import sys
dataFolder = '../../../../data/fields/'
sys.path.append(dataFolder)
from Auxiliary.helper import *
import xarray as xr 
import rioxarray as rio 
%matplotlib inline
from pathlib import Path
from rocksdbutils_copy import *
import cv2
from math import ceil as mceil


    

img_size = 128
## create metadata file
metadata = {
    'inputs': {
        'inputs_shape': (4, 6, img_size, img_size),  
        'inputs_dtype': np.float32     
    },
    'labels': {
        'labels_shape': (4, img_size, img_size),          
        'labels_dtype': np.float32
    }
}

## define function to load imgs and labs
def names2array_function(names):

    variables2use=['B2','B3','B4','B8']#,'NDVI']

    image_path, label_path = names
    # load image
    img = xr.open_dataset(image_path)
    image = np.concatenate([img[var].values[None] for var in variables2use],0)
    
    # load label
    ds = xr.open_dataset(label_path)
    label = np.asarray(ds['band_data'].values)
    label[np.isnan(label)] = 0
    return [image, label] 

def names2array_function16(names):

    variables2use=['B2','B3','B4','B8']#,'NDVI']

    image_path, label_path = names
    # load image
    img = xr.open_dataset(image_path)
    image = np.concatenate([img[var].values[None] for var in variables2use],0)
    
    # load label
    ds = xr.open_dataset(label_path)
    label = np.asarray(ds['band_data'].values)
    label[np.isnan(label)] = 0
    return [image.astype(np.float16), label.astype(np.float16)]  

#creator_transform = DatasetCreationTransform(NClasses=2, creation_transform=AI4BNormal_S2())

In [3]:
## create list of images and labels

folders = ['LU'] #['AT', 'ES', 'FR', 'NL', 'SE', 'SI']
imgs = [getFilelist(dataFolder + 'ai4boundaries/sentinel2/images/' + folder, '.nc') for folder in folders]
imgs = [img for list in imgs for img in list]
labs = [getFilelist(dataFolder + 'ai4boundaries/sentinel2/masks/' + folder, '.tif') for folder in folders]
labs = [lab for list in labs for lab in list]
print(len(imgs), len(labs))
imgs.sort()
labs.sort()

aa = checkemptyNC(labs)
# exclude images with empty labels and their labels
labs = [lab for i, lab in enumerate(labs) if i not in aa]
imgs = [img for i, img in enumerate(imgs) if i not in aa]

print(len(imgs), len(labs))

132 132
132 132


In [4]:
img_lab_paths = []
for i in range(len(imgs)):
    img_lab_paths.append((imgs[i], labs[i]))

# img_lab_paths = img_lab_paths[:2]


# aa = names2array_function(img_lab_paths[4])[0]
# aa.astype(np.int16)

## create db
output_dir = dataFolder + 'output/rocks_db/LU_transformed.db'
os.makedirs(output_dir, exist_ok=True)

rasters2rocks = Rasters2RocksDB(
    lstOfTuplesNames=img_lab_paths,            
    names2raster_function=names2array_function,  
    metadata=metadata,                       
    flname_prefix_save=output_dir,           
    batch_size=2,
    transformT=TrainingTransform_for_rocks_Train(),
    transformV=TrainingTransform_for_rocks_Valid(),
    #stride_divisor=2,                           
    #train_split=0.8,                         
    Filter=img_size,
    split_type='sequential'                  
)

rasters2rocks.create_dataset()

Creating databases in location:../../../../data/fields/output/rocks_db/LU_transformed.db
Database train::../../../../data/fields/output/rocks_db/LU_transformed.db/train.db
Database valid::../../../../data/fields/output/rocks_db/LU_transformed.db/valid.db
----------------------------
Processing:: 1/132 Tuple
----------------------------
Processing File:../../../../data/fields/ai4boundaries/sentinel2/images/LU/LU_100_S2_10m_256.nc
Processing File:../../../../data/fields/ai4boundaries/sentinel2/masks/LU/LU_100_S2label_10m_256.tif
****************************
[############################################################] 4/4
----------------------------
Processing:: 2/132 Tuple
----------------------------
Processing File:../../../../data/fields/ai4boundaries/sentinel2/images/LU/LU_101_S2_10m_256.nc
Processing File:../../../../data/fields/ai4boundaries/sentinel2/masks/LU/LU_101_S2label_10m_256.tif
****************************
[############################################################] 4

In [ ]:
# play around with RasterMaskIterableInMemoryFP
train_split = 0.9
myiterset = RasterMaskIterableInMemoryFP(names2array_function(img_lab_paths[0]),
                                                   Filter = img_size,
                                                   stride_divisor=2 ,
                                                   transform=None,#TrainingTransform_for_rocks(mode='train'),
                                                   batch_size=2,
                                                   batch_dimension=False)

plotter(names2array_function(img_lab_paths[0])[0][:,4,:,:])
nbset = myiterset.get_len_batch_set()
train_spliti = int(train_split*nbset)

print(nbset)    
print(train_spliti)
bL = []
for idx2 in progressbar(range(nbset)):
   batch = myiterset.get_batch(idx2)
   bL.append(batch)

    #self.write_split_strategy(idx2,train_spliti,batch)
bL[0][0][0][0,0,0,:,:]

plotter_batchlist(bL)

In [ ]:
# check out Test.dbs
train_dataset = RocksDBDataset('/home/output/rocks_db/Test5.db/train.db')
contiT = []
contilabel = []
contiV = []
for i in range(6,12,1):
    contiT.append(train_dataset[i][0])
    contilabel.append(train_dataset[i][1])

plotter4D(contiT, 2, 3)

valid_dataset = RocksDBDataset('/home/output/rocks_db/Test5.db/valid.db')
for i in range(2,4,1):
    contiV.append(valid_dataset[i][0])

plotter4D(contiV, 1, 2)

In [ ]:
#print(conti[0])
a = TrainingTransform_for_rocks_Train()

print(contiT[0].shape)
print(contilabel[0].shape)

b = a(contiT[0][:,:,:,:], contilabel[0][:,:,:])

print(contiT[0][0,0,0,0])
print(b[0][0,0,0,0,])

plotter(contiT[0][0])
plotter(b[0][0])

